# FlyThruGateAvitary curriculum checks

Goals:
- Create an env with `use_curriculum=True` and inspect observation/action spaces.
- Reset and verify the spawn point per curriculum level.
- Run a few random steps to ensure the env runs.
- Compare actual spawn vs expected position for each level (optional GUI preview).


In [ ]:
from gym_pybullet_drones.envs.FlyThruGateAvitary import FlyThruGateAvitary
from gym_pybullet_drones.utils.enums import ObservationType, ActionType
import numpy as np


In [ ]:
# Create env with curriculum enabled
env = FlyThruGateAvitary(
    obs=ObservationType('kin'),
    act=ActionType('rpm'),
    gui=False,
    record=False,
    use_curriculum=True,
    curriculum_level=0,
    max_curriculum_level=5,
)

print('Observation space:', env.observation_space)
print('Action space:', env.action_space)
print(f'Curriculum level: {env.curriculum_level}/{env.max_curriculum_level}')


In [ ]:
# Reset once to inspect spawn point and info
obs, info = env.reset()
obs_arr = np.array(obs)
print('Initial obs shape:', obs_arr.shape)
print('Start position (xyz):', obs_arr[:3])
print('Info keys:', list(info.keys()))


In [ ]:
# Run a few random steps as a quick smoke test
for step in range(5):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    obs_arr = np.array(obs)
    print(f'Step {step}: reward={reward:.3f}, terminated={terminated}, truncated={truncated}')
    print('  pos:', obs_arr[:3], 'vel:', obs_arr[3:6], 'rpy:', obs_arr[6:9])


In [ ]:
def preview_spawn_levels(max_level=5, num_steps=20, use_gui=False, sleep_sec=0.05):
    """Print spawn position for each level and compare to the expected offset.

    If GUI is enabled, each level opens a PyBullet window and runs zero action for a few steps.
    """
    gate_center = np.array([0.0, -1.0, 0.5])  # per reset(): gate + [0,0,0.5]

    for lvl in range(max_level + 1):
        sim_env = FlyThruGateAvitary(
            obs=ObservationType('kin'),
            act=ActionType('rpm'),
            gui=use_gui,
            record=False,
            use_curriculum=True,
            curriculum_level=lvl,
            max_curriculum_level=max_level,
        )

        obs, _ = sim_env.reset()
        obs_arr = np.array(obs)

        expected = gate_center + np.array([0.0, 0.1 + 0.15 * lvl, 0.0])
        delta = obs_arr[:3] - expected
        print(f'Level {lvl}: expected={expected}, start xyz={obs_arr[:3]}, delta={delta}')

        zero_action = np.zeros_like(sim_env.action_space.sample())
        for _ in range(num_steps):
            obs, reward, terminated, truncated, info = sim_env.step(zero_action)
            if use_gui:
                import time
                time.sleep(sleep_sec)
            if terminated or truncated:
                break
        sim_env.close()


In [ ]:
# Quick preview of all levels without GUI
preview_spawn_levels(max_level=env.max_curriculum_level, num_steps=5, use_gui=False)


## Visualize spawn with GUI
Uncomment the cell below to open PyBullet for each level (window closes after `num_steps`).


In [ ]:
# preview_spawn_levels(max_level=env.max_curriculum_level, num_steps=40, use_gui=True, sleep_sec=0.05)


In [ ]:
env.close()
